In [235]:
# Dependencies
import numpy as np
import pandas as pd
import requests
from census import Census
import gmaps

import warnings
warnings.filterwarnings('ignore')

# Census & gmaps API Keys
from config import (api_key, gkey)
c = Census(api_key, year=2020)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [236]:
# https://api.census.gov/data/2020/acs/acs1?get=NAME,B01001_001E&for=county:*
# https://api.census.gov/data/2020/acs/acs1?get=NAME,B01001_001E&for=county:*&in=state:*
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B01003_001E"), {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

census_pd.head()

,NAME,B01003_001E,state
0,Pennsylvania,12794885.0,42
1,California,39346023.0,06
2,West Virginia,1807426.0,54
3,Utah,3151239.0,49
4,New York,19514849.0,36


In [237]:
# Remove column name 'state'
census_pd = census_pd.drop(['state'], axis=1)
census_pd.head()

,NAME,B01003_001E
0,Pennsylvania,12794885.0
1,California,39346023.0
2,West Virginia,1807426.0
3,Utah,3151239.0
4,New York,19514849.0


In [238]:
census_pd.dtypes

NAME            object
B01003_001E    float64
dtype: object

In [239]:
census_pd.columns

Index(['NAME', 'B01003_001E'], dtype='object')

In [240]:
# Rename the column names

census_pd.rename(columns = {'NAME':'State', 'B01003_001E':'Population'}, inplace = True)
census_pd.head()

,State,Population
0,Pennsylvania,12794885.0
1,California,39346023.0
2,West Virginia,1807426.0
3,Utah,3151239.0
4,New York,19514849.0


In [241]:
# To convert from float to integer

census_pd['Population'] = census_pd['Population'].astype(int)
census_pd.head()

,State,Population
0,Pennsylvania,12794885
1,California,39346023
2,West Virginia,1807426
3,Utah,3151239
4,New York,19514849


In [242]:
census_pd.loc[census_pd['State'] == 'Hawaii']

,State,Population
21,Hawaii,1420074


In [243]:
# Importing mortality data file

csv_path = 'https://raw.githubusercontent.com/uzmabb182/Data602_Assignments/main/Final_Project_Data_602/resources/posgres_states_df.csv'
states_df = pd.read_csv(csv_path, encoding="utf-8")
states_df.head(5)

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [244]:
# When column names are different
df=pd.merge(census_pd,states_df, left_on='State', right_on='State', how='left')
df.head()

,State,Population,Abbreviation
0,Pennsylvania,12794885,PA
1,California,39346023,CA
2,West Virginia,1807426,WV
3,Utah,3151239,UT
4,New York,19514849,NY


In [245]:
# Importing mortality data file

csv_path = 'https://raw.githubusercontent.com/uzmabb182/Data602_Assignments/main/Final_Project_Data_602/resources/posgres_modality_df.csv'
modality_df = pd.read_csv(csv_path, encoding="utf-8")
modality_df

,learning_modality,year,state,student_count
0,Hybrid,2021,AK,8033
1,Hybrid,2021,AL,5863
2,Hybrid,2021,AR,1354
3,Hybrid,2021,AZ,735
4,Hybrid,2021,BI,414
...,...,...,...,...
271,Remote,2022,SD,2135
272,Remote,2022,TX,12263
273,Remote,2022,UT,660
274,Remote,2022,WA,232


In [246]:
# When column names are different
population_df=pd.merge(modality_df,df, left_on='state', right_on='Abbreviation', how='left')
population_df

,learning_modality,year,state,student_count,State,Population,Abbreviation
0,Hybrid,2021,AK,8033,Alaska,736990.0,AK
1,Hybrid,2021,AL,5863,Alabama,4893186.0,AL
2,Hybrid,2021,AR,1354,Arkansas,3011873.0,AR
3,Hybrid,2021,AZ,735,Arizona,7174064.0,AZ
4,Hybrid,2021,BI,414,NaN,NaN,NaN
...,...,...,...,...,...,...,...
271,Remote,2022,SD,2135,South Dakota,879336.0,SD
272,Remote,2022,TX,12263,Texas,28635442.0,TX
273,Remote,2022,UT,660,Utah,3151239.0,UT
274,Remote,2022,WA,232,Washington,7512465.0,WA


In [247]:
population_df['State'].values.tolist()

['Alaska',
 'Alabama',
 'Arkansas',
 'Arizona',
 nan,
 'California',
 'Colorado',
 'Connecticut',
 'District of Columbia',
 'Delaware',
 'Florida',
 'Georgia',
 'Iowa',
 'Idaho',
 'Illinois',
 'Indiana',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Massachusetts',
 'Maryland',
 'Maine',
 'Michigan',
 'Minnesota',
 'Missouri',
 'Mississippi',
 'Montana',
 'North Carolina',
 'Nebraska',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'Nevada',
 'New York',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Virginia',
 'Vermont',
 'Washington',
 'Wisconsin',
 'West Virginia',
 'Alaska',
 'Alabama',
 'Arkansas',
 'Arizona',
 nan,
 'California',
 'Colorado',
 'Connecticut',
 'District of Columbia',
 'Delaware',
 'Georgia',
 'Iowa',
 'Idaho',
 'Illinois',
 'Indiana',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Massachusetts',
 'Maryland',
 'Maine',
 'Michigan',
 'Minnesota',
 'Missouri',
 'Mississippi',
 'Montana'

In [248]:
population_df = population_df.dropna()
population_df

,learning_modality,year,state,student_count,State,Population,Abbreviation
0,Hybrid,2021,AK,8033,Alaska,736990.0,AK
1,Hybrid,2021,AL,5863,Alabama,4893186.0,AL
2,Hybrid,2021,AR,1354,Arkansas,3011873.0,AR
3,Hybrid,2021,AZ,735,Arizona,7174064.0,AZ
5,Hybrid,2021,CA,3320,California,39346023.0,CA
...,...,...,...,...,...,...,...
271,Remote,2022,SD,2135,South Dakota,879336.0,SD
272,Remote,2022,TX,12263,Texas,28635442.0,TX
273,Remote,2022,UT,660,Utah,3151239.0,UT
274,Remote,2022,WA,232,Washington,7512465.0,WA


In [249]:
# converting 'Weight' from float to int
population_df['Population'] = population_df['Population'].astype(int)
population_df

,learning_modality,year,state,student_count,State,Population,Abbreviation
0,Hybrid,2021,AK,8033,Alaska,736990,AK
1,Hybrid,2021,AL,5863,Alabama,4893186,AL
2,Hybrid,2021,AR,1354,Arkansas,3011873,AR
3,Hybrid,2021,AZ,735,Arizona,7174064,AZ
5,Hybrid,2021,CA,3320,California,39346023,CA
...,...,...,...,...,...,...,...
271,Remote,2022,SD,2135,South Dakota,879336,SD
272,Remote,2022,TX,12263,Texas,28635442,TX
273,Remote,2022,UT,660,Utah,3151239,UT
274,Remote,2022,WA,232,Washington,7512465,WA


In [250]:
population_df["student_count_per_10k"] = (population_df["student_count"] / population_df["Population"])* 10000
population_df

,learning_modality,year,state,student_count,State,Population,Abbreviation,student_count_per_10k
0,Hybrid,2021,AK,8033,Alaska,736990,AK,108.997408
1,Hybrid,2021,AL,5863,Alabama,4893186,AL,11.981968
2,Hybrid,2021,AR,1354,Arkansas,3011873,AR,4.495541
3,Hybrid,2021,AZ,735,Arizona,7174064,AZ,1.024524
5,Hybrid,2021,CA,3320,California,39346023,CA,0.843796
...,...,...,...,...,...,...,...,...
271,Remote,2022,SD,2135,South Dakota,879336,SD,24.279684
272,Remote,2022,TX,12263,Texas,28635442,TX,4.282455
273,Remote,2022,UT,660,Utah,3151239,UT,2.094414
274,Remote,2022,WA,232,Washington,7512465,WA,0.308820


In [251]:
# Remove column name 'state'
population_df = population_df.drop(['state'], axis=1)
population_df

,learning_modality,year,student_count,State,Population,Abbreviation,student_count_per_10k
0,Hybrid,2021,8033,Alaska,736990,AK,108.997408
1,Hybrid,2021,5863,Alabama,4893186,AL,11.981968
2,Hybrid,2021,1354,Arkansas,3011873,AR,4.495541
3,Hybrid,2021,735,Arizona,7174064,AZ,1.024524
5,Hybrid,2021,3320,California,39346023,CA,0.843796
...,...,...,...,...,...,...,...
271,Remote,2022,2135,South Dakota,879336,SD,24.279684
272,Remote,2022,12263,Texas,28635442,TX,4.282455
273,Remote,2022,660,Utah,3151239,UT,2.094414
274,Remote,2022,232,Washington,7512465,WA,0.308820


In [252]:
population_df.columns = population_df.columns.str.lower()
population_df

,learning_modality,year,student_count,state,population,abbreviation,student_count_per_10k
0,Hybrid,2021,8033,Alaska,736990,AK,108.997408
1,Hybrid,2021,5863,Alabama,4893186,AL,11.981968
2,Hybrid,2021,1354,Arkansas,3011873,AR,4.495541
3,Hybrid,2021,735,Arizona,7174064,AZ,1.024524
5,Hybrid,2021,3320,California,39346023,CA,0.843796
...,...,...,...,...,...,...,...
271,Remote,2022,2135,South Dakota,879336,SD,24.279684
272,Remote,2022,12263,Texas,28635442,TX,4.282455
273,Remote,2022,660,Utah,3151239,UT,2.094414
274,Remote,2022,232,Washington,7512465,WA,0.308820


In [253]:
population_df.loc[modality_df['learning_modality'] == 'In Person']

,learning_modality,year,student_count,state,population,abbreviation,student_count_per_10k
98,In Person,2021,3275,Alaska,736990,AK,44.437509
99,In Person,2021,5477,Alabama,4893186,AL,11.193116
100,In Person,2021,2102,Arkansas,3011873,AR,6.979046
101,In Person,2021,2401,Arizona,7174064,AZ,3.346778
103,In Person,2021,5509,California,39346023,CA,1.400142
...,...,...,...,...,...,...,...
197,In Person,2022,1373,Vermont,624340,VT,21.991223
198,In Person,2022,4065,Washington,7512465,WA,5.411007
199,In Person,2022,2088,Wisconsin,5806975,WI,3.595676
200,In Person,2022,5204,West Virginia,1807426,WV,28.792327


In [254]:
population_df.loc[population_df['state'] == 'Hawaii']

,learning_modality,year,student_count,state,population,abbreviation,student_count_per_10k


In [255]:
population_df.dtypes

learning_modality         object
year                       int64
student_count              int64
state                     object
population                 int32
abbreviation              object
student_count_per_10k    float64
dtype: object

In [256]:
population_df.loc[population_df['abbreviation'] == 'HI']

,learning_modality,year,student_count,state,population,abbreviation,student_count_per_10k


In [257]:
# Save as a csv for Tableau dashboard
# Note to avoid any issues later, use encoding="utf-8"
population_df.to_csv("resources/tableau_modality_df.csv", encoding="utf-8", index=False)

In [258]:
# Importing covid data file

csv_path = 'https://raw.githubusercontent.com/uzmabb182/Data602_Assignments/main/Final_Project_Data_602/resources/posgres_covid_df.csv'
covid_df = pd.read_csv(csv_path, encoding="utf-8")
covid_df

,state,cases,deaths,year
0,Alabama,365747,4872,2021
1,Alaska,46740,198,2021
2,Arizona,530267,9015,2021
3,Arkansas,229442,3711,2021
4,California,2345811,26236,2021
...,...,...,...,...
38595,Virginia,2143804,22495,2022
38596,Washington,1854190,14775,2022
38597,West Virginia,613165,7714,2022
38598,Wisconsin,1921427,15651,2022


In [259]:
# When column names are different
cases_df=pd.merge(covid_df,df, left_on='state', right_on='State', how='left')
cases_df

,state,cases,deaths,year,State,Population,Abbreviation
0,Alabama,365747,4872,2021,Alabama,4893186.0,AL
1,Alaska,46740,198,2021,Alaska,736990.0,AK
2,Arizona,530267,9015,2021,Arizona,7174064.0,AZ
3,Arkansas,229442,3711,2021,Arkansas,3011873.0,AR
4,California,2345811,26236,2021,California,39346023.0,CA
...,...,...,...,...,...,...,...
38595,Virginia,2143804,22495,2022,Virginia,8509358.0,VA
38596,Washington,1854190,14775,2022,Washington,7512465.0,WA
38597,West Virginia,613165,7714,2022,West Virginia,1807426.0,WV
38598,Wisconsin,1921427,15651,2022,Wisconsin,5806975.0,WI


In [260]:
# converting 'Weight' from float to int
cases_df = cases_df.dropna()
cases_df['Population'] = cases_df['Population'].astype(int)
cases_df

,state,cases,deaths,year,State,Population,Abbreviation
0,Alabama,365747,4872,2021,Alabama,4893186,AL
1,Alaska,46740,198,2021,Alaska,736990,AK
2,Arizona,530267,9015,2021,Arizona,7174064,AZ
3,Arkansas,229442,3711,2021,Arkansas,3011873,AR
4,California,2345811,26236,2021,California,39346023,CA
...,...,...,...,...,...,...,...
38595,Virginia,2143804,22495,2022,Virginia,8509358,VA
38596,Washington,1854190,14775,2022,Washington,7512465,WA
38597,West Virginia,613165,7714,2022,West Virginia,1807426,WV
38598,Wisconsin,1921427,15651,2022,Wisconsin,5806975,WI


In [261]:
cases_df["cases_per_10k"] = (cases_df["cases"] / cases_df["Population"]) * 10000
cases_df["deaths_per_10k"] = (cases_df["deaths"] / cases_df["Population"]) * 10000
cases_df

,state,cases,deaths,year,State,Population,Abbreviation,cases_per_10k,deaths_per_10k
0,Alabama,365747,4872,2021,Alabama,4893186,AL,747.461879,9.956703
1,Alaska,46740,198,2021,Alaska,736990,AK,634.201278,2.686604
2,Arizona,530267,9015,2021,Arizona,7174064,AZ,739.144507,12.566099
3,Arkansas,229442,3711,2021,Arkansas,3011873,AR,761.791749,12.321237
4,California,2345811,26236,2021,California,39346023,CA,596.200282,6.668018
...,...,...,...,...,...,...,...,...,...
38595,Virginia,2143804,22495,2022,Virginia,8509358,VA,2519.348698,26.435602
38596,Washington,1854190,14775,2022,Washington,7512465,WA,2468.151266,19.667313
38597,West Virginia,613165,7714,2022,West Virginia,1807426,WV,3392.476372,42.679479
38598,Wisconsin,1921427,15651,2022,Wisconsin,5806975,WI,3308.826024,26.952071


In [262]:
# Remove column name 'state'
cases_df = cases_df.drop(['state'], axis=1)
cases_df

,cases,deaths,year,State,Population,Abbreviation,cases_per_10k,deaths_per_10k
0,365747,4872,2021,Alabama,4893186,AL,747.461879,9.956703
1,46740,198,2021,Alaska,736990,AK,634.201278,2.686604
2,530267,9015,2021,Arizona,7174064,AZ,739.144507,12.566099
3,229442,3711,2021,Arkansas,3011873,AR,761.791749,12.321237
4,2345811,26236,2021,California,39346023,CA,596.200282,6.668018
...,...,...,...,...,...,...,...,...
38595,2143804,22495,2022,Virginia,8509358,VA,2519.348698,26.435602
38596,1854190,14775,2022,Washington,7512465,WA,2468.151266,19.667313
38597,613165,7714,2022,West Virginia,1807426,WV,3392.476372,42.679479
38598,1921427,15651,2022,Wisconsin,5806975,WI,3308.826024,26.952071


In [263]:
cases_df.columns = cases_df.columns.str.lower()
cases_df.dtypes

cases               int64
deaths              int64
year                int64
state              object
population          int32
abbreviation       object
cases_per_10k     float64
deaths_per_10k    float64
dtype: object

In [264]:
# Save as a csv for Tableau dashboard
# Note to avoid any issues later, use encoding="utf-8"
cases_df.to_csv("resources/tableau_covid_df.csv", encoding="utf-8", index=False)